# Modeling

In [1]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("train_ready_for_model.csv")

In [3]:
test = pd.read_csv("test_ready_for_model.csv")

In [4]:
result = test[["User_ID"]]

In [5]:
train.columns

Index([u'User_ID', u'Browser_Chrome', u'Browser_Edge', u'Browser_Firefox',
       u'Browser_IE', u'Browser_Opera', u'Browser_Safari', u'Device_Desktop',
       u'Device_Mobile', u'Device_Tablet', u'Is_Response', u'sentiment'],
      dtype='object')

In [6]:
result["Is_Response"] = train.Is_Response

In [7]:
result.to_csv("res1.csv",index=False)

In [40]:
final=pd.read_csv('res1.csv')
final.Is_Response = final.Is_Response.astype(str)
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('1.0','happy'))
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('0.0','not happy'))
#final.head()
final=final[["User_ID","Is_Response"]]
final.to_csv("res1.csv",index=False)

In [9]:
target = 'Is_Response'
IDcol = ['User_ID']
from sklearn import model_selection, metrics
from sklearn.metrics import classification_report

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
    
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    
    #CrossValidation:
    cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='accuracy')
    print("10-fold cross validation average accuracy: %.3f" % (cv_train_score.mean()))
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    print(classification_report(dtrain[target].values, dtrain_predictions))
    #print("Train CV score : ", cv_train_score)
    #print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
        
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [10]:
from sklearn.linear_model import LogisticRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
print("predictors : ",predictors)
# print predictors
alg1 = LogisticRegression()
modelfit(alg1, train, test, predictors, target, IDcol, 'res2.csv')
##coef1 = pd.Series(alg1.coef_, predictors).sort_values()
##coef1.plot(kind='bar', title='Model Coefficients')

('predictors : ', ['Browser_Chrome', 'Browser_Edge', 'Browser_Firefox', 'Browser_IE', 'Browser_Opera', 'Browser_Safari', 'Device_Desktop', 'Device_Mobile', 'Device_Tablet', 'sentiment'])
10-fold cross validation average accuracy: 0.741

Model Report
             precision    recall  f1-score   support

        0.0       0.64      0.43      0.52     12411
        1.0       0.77      0.88      0.82     26521

avg / total       0.73      0.74      0.73     38932



In [39]:
final=pd.read_csv('res2.csv')
final.Is_Response = final.Is_Response.astype(str)
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('1.0','happy'))
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('0.0','not happy'))
#final.head()
final=final[["User_ID","Is_Response"]]
final.to_csv("res2.csv",index=False)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
for i in range(1,5):
    alg1 = KNeighborsClassifier(n_neighbors=i)
    modelfit(alg1, train, test, predictors, target, IDcol, 'res3.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

10-fold cross validation average accuracy: 0.663

Model Report
             precision    recall  f1-score   support

        0.0       0.75      0.75      0.75     12411
        1.0       0.88      0.89      0.88     26521

avg / total       0.84      0.84      0.84     38932

10-fold cross validation average accuracy: 0.622

Model Report
             precision    recall  f1-score   support

        0.0       0.58      0.89      0.70     12411
        1.0       0.93      0.70      0.80     26521

avg / total       0.82      0.76      0.77     38932

10-fold cross validation average accuracy: 0.694

Model Report
             precision    recall  f1-score   support

        0.0       0.70      0.60      0.65     12411
        1.0       0.83      0.88      0.85     26521

avg / total       0.78      0.79      0.79     38932

10-fold cross validation average accuracy: 0.680

Model Report
             precision    recall  f1-score   support

        0.0       0.61      0.72      0.66     12

In [38]:
final=pd.read_csv('res3.csv')
final.Is_Response = final.Is_Response.astype(str)
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('1.0','happy'))
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('0.0','not happy'))
#final.head()
final=final[["User_ID","Is_Response"]]
final.to_csv("res3.csv",index=False)

In [14]:
from sklearn.ensemble import RandomForestClassifier
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = RandomForestClassifier(n_estimators=260,max_depth=15, min_samples_leaf=100)
modelfit(alg1, train, test, predictors, target, IDcol, 'res4.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

10-fold cross validation average accuracy: 0.740

Model Report
             precision    recall  f1-score   support

        0.0       0.66      0.42      0.51     12411
        1.0       0.77      0.90      0.83     26521

avg / total       0.73      0.75      0.73     38932



In [37]:
final=pd.read_csv('res4.csv')
final.Is_Response = final.Is_Response.astype(str)
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('1.0','happy'))
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('0.0','not happy'))
#final.head()
final=final[["User_ID","Is_Response"]]
final.to_csv("res4.csv",index=False)

In [16]:
from sklearn.ensemble import AdaBoostClassifier
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = AdaBoostClassifier(n_estimators=260)
modelfit(alg1, train, test, predictors, target, IDcol, 'res5.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

10-fold cross validation average accuracy: 0.743

Model Report
             precision    recall  f1-score   support

        0.0       0.64      0.45      0.53     12411
        1.0       0.77      0.88      0.83     26521

avg / total       0.73      0.74      0.73     38932



In [36]:
final=pd.read_csv('res5.csv')
final.Is_Response = final.Is_Response.astype(str)
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('1.0','happy'))
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('0.0','not happy'))
#final.head()
final=final[["User_ID","Is_Response"]]
final.to_csv("res5.csv",index=False)

In [18]:
from sklearn.tree import DecisionTreeClassifier
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = DecisionTreeClassifier(max_depth=15, min_samples_leaf=100)
modelfit(alg1, train, test, predictors, target, IDcol, 'res6.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

10-fold cross validation average accuracy: 0.740

Model Report
             precision    recall  f1-score   support

        0.0       0.65      0.46      0.54     12411
        1.0       0.78      0.88      0.83     26521

avg / total       0.74      0.75      0.74     38932



In [35]:
final=pd.read_csv('res6.csv')
final.Is_Response = final.Is_Response.astype(str)
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('1.0','happy'))
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('0.0','not happy'))
#final.head()
final=final[["User_ID","Is_Response"]]
final.to_csv("res6.csv",index=False)

In [20]:
import xgboost as xgb
predictors = [x for x in train.columns if x not in [target]+IDcol]
##predictors = coef5.index[:7]
alg5 = xgb.XGBClassifier(max_depth=10, n_estimators=260, learning_rate=0.05)
modelfit(alg5, train, test, predictors, target, IDcol, 'res7.csv')
##coef5 = pd.Series(alg5.feature_importances_, predictors).sort_values(ascending=False)
##coef5.plot(kind='bar', title='Feature Importances')

C:\Users\user\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


10-fold cross validation average accuracy: 0.737

Model Report
             precision    recall  f1-score   support

        0.0       0.72      0.52      0.60     12411
        1.0       0.80      0.90      0.85     26521

avg / total       0.77      0.78      0.77     38932



In [34]:
final=pd.read_csv('res7.csv')
final.Is_Response = final.Is_Response.astype(str)
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('1.0','happy'))
final.Is_Response=final.Is_Response.apply( lambda x: x.replace('0.0','not happy'))
final=final[["User_ID","Is_Response"]]
final.to_csv("res7.csv",index=False)